<a href="https://colab.research.google.com/github/bhstoller/multi-agent-customer-service/blob/main/agent_to_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-Agent Customer Service System with A2A and MCP**

This notebook demonstrates agent-to-agent (A2A) communication using Google ADK with MCP (Model Context Protocol) tools.

## System Components
- **Customer Data Agent**: Interfaces with MCP server to fetch/update customer data
- **Support Agent**: Handles customer support queries and troubleshooting
- **Router**: Orchestrates communication between agents based on query type

## Environment Setup

Installing required packages and dependencies.

In [1]:
!pip install uv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 43.2 MB/s eta 0:00:00


In [2]:
# Install required packages
%pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0 python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.5/260.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## Import Dependencies

Loading required libraries for A2A communication, MCP integration, and server setup.

In [3]:
import sys
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver

class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [4]:
import sys
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver
import asyncio
import logging
import os
import sys
import threading
import time
from typing import Any
import httpx
import nest_asyncio
import uvicorn
from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_toolset import StreamableHTTPConnectionParams
import re
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


## Configuration

Setting up Google Cloud and MCP server configuration.

In [5]:
# Set Google Cloud Configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'app-ai-a5'
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'

load_dotenv()
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("a5-key")

print('Environment variables configured:')
print(f'GOOGLE_GENAI_USE_VERTEXAI: {os.environ["GOOGLE_GENAI_USE_VERTEXAI"]}')
print(f'GOOGLE_CLOUD_PROJECT: {os.environ["GOOGLE_CLOUD_PROJECT"]}')
print(f'GOOGLE_CLOUD_LOCATION: {os.environ["GOOGLE_CLOUD_LOCATION"]}')
print("GOOGLE_API_KEY loaded =", os.environ["GOOGLE_API_KEY"] is not None)

Environment variables configured:
GOOGLE_GENAI_USE_VERTEXAI: FALSE
GOOGLE_CLOUD_PROJECT: app-ai-a5
GOOGLE_CLOUD_LOCATION: us-central1
GOOGLE_API_KEY loaded = True


In [6]:
if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

In [7]:
MCP_SERVER_URL = userdata.get('MCP_SERVER_URL')
print(MCP_SERVER_URL)

https://polar-nonsolubly-madden.ngrok-free.dev/mcp


## A2A Communication Logger

Custom logger to track agent-to-agent communication flow.

In [8]:
# Configure basic logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    force=True
)

# Add A2A Communication Logger
print("Setting up A2A Communication Logger...")

class A2ALogger:
    """Tracks agent-to-agent communication"""

    def __init__(self):
        self.request_count = 0

    def log_query(self, query: str):
        """Log user query"""
        self.request_count += 1
        print("\n" + "="*70)
        print(f"USER QUERY")
        print("="*70)
        print(f"Query: {query}")
        print("="*70 + "\n")

    def log_response(self, response: str):
        """Log final response"""
        print("\n" + "="*70)
        print(f"FINAL RESPONSE")
        print("="*70)
        print(response)
        print("="*70 + "\n")

# Initialize logger
a2a_logger = A2ALogger()
print("A2A Logger Ready!")

Setting up A2A Communication Logger...
A2A Logger Ready!


## Agent Initialization

Defining the three specialized agents that form the multi-agent customer service system:


1.   Customer Data Agent
2.   Support Agent
3.   Router (Orchestrator) Agent

Each agent has specific responsibilities and logging capabilities to track A2A communication.

### Add Customer Data Agent

Interfaces directly with the MCP server to perform all customer data operations including retrieval, updates, and ticket management. This agent announces all MCP tool calls with `[CUSTOMER_DATA_AGENT]:` prefix for visibility into data operations.

In [9]:
customer_data_agent = Agent(
    model="gemini-2.0-flash",
    name="customer_data_agent",
    tools=[ MCPToolset(
        connection_params=StreamableHTTPConnectionParams(
            url=MCP_SERVER_URL
            )
        )
    ],
    instruction="""
    You are the Customer Data Agent.

    IMPORTANT LOGGING: Begin EVERY response with:
    [CUSTOMER_DATA_AGENT]: <brief description of what you're doing>

    Your role is to interface with the MCP server and perform EXACT data operations.

    REQUIRED RULES:
    - ALWAYS announce your action first: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID X
    - ALWAYS call an MCP tool for any operation involving customer data.
    - NEVER invent fields, values, or IDs.
    - ALWAYS return valid JSON after your announcement.
    - If a customer does not exist, return:
      {"error": "Customer ID not found", "customer_id": <id>}
    - For successful lookups, return:
      {"customer": { ...mcp result... }}

    AVAILABLE OPERATIONS:
    - get_customer(customer_id)
    - list_customers(status, limit)
    - update_customer(customer_id, data)
    - create_ticket(customer_id, issue, priority)
    - get_customer_history(customer_id)

    ADDITIONAL RULES:
    - When updating a customer, preserve fields not being modified.
    - When creating a ticket, always include created_at returned by MCP.
    - When listing customers, return array of customer objects.

    ALWAYS start with [CUSTOMER_DATA_AGENT]: then provide JSON.
    """
)
print("Customer Data Agent created with logging!")

Customer Data Agent created with logging!


In [10]:
customer_data_agent_card = AgentCard(
    name='Customer Data Agent',
    url='http://127.0.0.1:10020',
    description='Fetches and updates customer data using MCP tools',
    version='1.0',
    capabilities=AgentCapabilities(streaming=False),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_data_access',
            name='Customer Data Access',
            description='Retrieve and update customer records',
            tags=['customer', 'data', 'database', 'lookup'],
            examples=[
                'Get customer 42',
                'Update customer 5 email',
                'Show all customers',
            ],
        )
    ],
)


In [11]:
remote_customer_data_agent = RemoteA2aAgent(
    name='customer_data_agent',
    description='Remote wrapper for Customer Data Agent',
    agent_card=f'http://127.0.0.1:10020{AGENT_CARD_WELL_KNOWN_PATH}',  # ← FIXED
)

### Add Support Agent

Handles customer support queries, troubleshooting, and escalations. Routes pure data retrieval requests to the Customer Data Agent, and requests customer context when needed to provide personalized support. Logs all actions with `[SUPPORT_AGENT]:` prefix.

In [12]:
support_agent = Agent(
    model="gemini-2.0-flash-exp",
    name="support_agent",
    instruction="""
    You are the Support Agent.

    IMPORTANT LOGGING: Begin EVERY response with:
    [SUPPORT_AGENT]: <brief description of what you're doing>

    CRITICAL RULE: If the user asks for customer INFORMATION/DATA (get, retrieve, show, lookup),
    you MUST respond with:

    [SUPPORT_AGENT]: This is a data retrieval request, not a support question.
    PASS_TO_CUSTOMER_DATA_AGENT

    Only handle SUPPORT questions like:
    - "How do I reset my password?"
    - "I need help with my account"
    - "I'm having trouble logging in"

    If the query is asking for customer data/information/records, say:
    [SUPPORT_AGENT]: PASS_TO_CUSTOMER_DATA_AGENT

    If customer-specific data is required for a SUPPORT question, respond with:
    [SUPPORT_AGENT]: Requesting customer data from router
    {
      "needs_customer_data": true,
      "reason": "<why>",
      "requested_fields": ["email", "tickets", "status", ...]
    }

    When the Router provides customer data for a support question:
    - Announce: [SUPPORT_AGENT]: Processing support request with customer context
    - Use only those fields
    - Provide a natural-language support answer

    Your job: troubleshooting, escalation, and answering support questions ONLY.
    For data retrieval queries, defer to customer data agent.

    ALWAYS start responses with [SUPPORT_AGENT]:
    """,
)
print("Support Agent updated with better routing logic!")

Support Agent updated with better routing logic!


In [13]:
support_agent_card = AgentCard(
    name='Support Agent',
    url='http://127.0.0.1:10021',   # FIXED
    description='Handles general support questions and escalates to customer data agent when needed',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_support',
            name='Customer Support',
            description='Provides general support responses',
            tags=['support', 'help', 'troubleshooting'],
            examples=[
                'How do I reset my password?',
                'What is your refund policy?',
                'I need help with my account.',
            ],
        )
    ],
)


In [14]:
remote_support_agent = RemoteA2aAgent(
    name='support_agent',
    description='Remote wrapper for Support Agent',
    agent_card=f'http://127.0.0.1:10021{AGENT_CARD_WELL_KNOWN_PATH}',  # FIXED
)

### Add Router (Orchestrator) Agent

Orchestrates the multi-agent system by coordinating between the Customer Data Agent and Support Agent. Built as a SequentialAgent that processes queries through specialized sub-agents to provide intelligent routing and response synthesis.

## Server Infrastructure

Setting up the A2A server infrastructure to host each agent on separate ports, enabling agent-to-agent communication via HTTP/JSON-RPC protocol.

### Server Setup Functions

Functions to create and run A2A servers for each agent.

In [15]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent.

    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card

    Returns:
        A2AStarletteApplication instance
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [16]:
# Apply nest_asyncio
nest_asyncio.apply()

# Store server tasks
server_tasks: list[asyncio.Task] = []

async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',
    )

    server = uvicorn.Server(config)
    await server.serve()

async def start_all_servers() -> None:
    """Start ONLY the Customer and Support servers."""
    tasks = [
        asyncio.create_task(
            run_agent_server(customer_data_agent, customer_data_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(support_agent, support_agent_card, 10021)
        ),
        # REMOVED: router_agent server on 10040
    ]

    await asyncio.sleep(2)

    print('Agent servers started!')
    print('   - Customer Data Agent: http://0.0.0.0:10020')
    print('   - Support Agent:       http://0.0.0.0:10021')

    try:
        await asyncio.Event().wait()
    except KeyboardInterrupt:
        print('Shutting down servers...')

def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())

server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

time.sleep(3)

Agent servers started!
   - Customer Data Agent: http://0.0.0.0:10020
   - Support Agent:       http://0.0.0.0:10021


In [17]:
print('Customer Data Agent Card:')
print(customer_data_agent_card)
print('Support Agent Card:')
print(support_agent_card)
print('Host Agent Card:')
print(router_agent_card)

Customer Data Agent Card:
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=False) default_input_modes=['text/plain'] default_output_modes=['application/json'] description='Fetches and updates customer data using MCP tools' documentation_url=None icon_url=None name='Customer Data Agent' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Retrieve and update customer records', examples=['Get customer 42', 'Update customer 5 email', 'Show all customers'], id='customer_data_access', input_modes=None, name='Customer Data Access', output_modes=None, security=None, tags=['customer', 'data', 'database', 'lookup'])] supports_authenticated_extended_card=None url='http://127.0.0.1:10020' version='1.0'
Support Agent Card:
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notific

NameError: name 'router_agent_card' is not defined

### A2A Client Setup

Client for sending messages to agent servers following the A2A protocol.

In [18]:
class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # Create the message object
            message_obj = create_text_message_object(content=message)

            # Send the message and collect responses
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # The response is a tuple - get the first element (Task object)
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # First element of the tuple

                # Extract text: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return 'No response received'

In [19]:
a2a_client = A2ASimpleClient()

In [34]:
import google.generativeai as genai
import os
import json
import asyncio

# 2. Define the Router Class (Updated for Batching)
class RouterOrchestrator:
    def __init__(self, a2a_client):
        self.client = a2a_client
        # Using the high-reasoning model for the "Brain"
        self.model = genai.GenerativeModel("gemini-2.0-flash-exp")

        # URLs for your running agents
        self.DATA_AGENT_URL = "http://localhost:10020"
        self.SUPPORT_AGENT_URL = "http://localhost:10021"

    async def call_agent(self, agent_name, query):
        """Helper to execute the A2A call to your running servers"""
        url = self.DATA_AGENT_URL if agent_name == "customer_data" else self.SUPPORT_AGENT_URL
        print(f"   >>> [A2A CALL] Connecting to {agent_name} at {url}...")

        try:
            # Use the existing a2a_client logic
            response = await self.client.create_task(url, query)
            return response
        except Exception as e:
            return f"Error calling agent: {str(e)}"

    async def process_query(self, user_query):
        print(f"\n{'='*60}\n USER QUERY: {user_query}\n{'='*60}")

        # Increased limit to handle larger data sets
        max_turns = 15

        # System prompt defines the Router's logic
        system_prompt = """
        You are the Router Agent (Orchestrator) for a customer service system.
        You have two specialized sub-agents you can call via A2A tools:

        1. "customer_data"
           - Capabilities: Get customer details, list customers, update records, get ticket history, create tickets.

        2. "support_agent"
           - Capabilities: General support advice, troubleshooting, escalation decisions.

        Your Goal: Answer the user's request by coordinating these agents.

        CRITICAL INSTRUCTION FOR LISTS:
        - If the user asks for a list of customers with specific conditions (e.g., "active customers with open tickets"), do NOT check them one by one.
        - **BATCH YOUR REQUESTS:**
          1. Call customer_data to list active customers.
          2. Send a **SINGLE** message to customer_data requesting ticket history for **ALL** the retrieved customer IDs at once (e.g. "Get history for customer IDs 4, 5, 8, 10...").
          3. Filter the results yourself based on the returned data.

        RESPONSE FORMAT:
        You must strictly return a JSON object in this format (no markdown formatting):
        {
            "thought": "Explanation of your reasoning",
            "action": "call_agent" OR "final_answer",
            "agent_name": "customer_data" OR "support_agent" (only if action is call_agent),
            "content": "The specific query string to send to that agent" OR "The final text response to the user"
        }
        """

        messages = [
            {"role": "user", "parts": [system_prompt + f"\n\nUser Query: {user_query}"]}
        ]

        for i in range(max_turns):
            try:
                response = await self.model.generate_content_async(messages)
                response_text = response.text
            except Exception as e:
                print(f"[ROUTER ERROR]: Gemini generation failed: {e}")
                return "System Error"

            # Clean up JSON
            clean_text = response_text.replace("```json", "").replace("```", "").strip()

            try:
                plan = json.loads(clean_text)
            except json.JSONDecodeError:
                print(f"[ROUTER ERROR]: Could not parse JSON plan. Raw: {clean_text}")
                messages.append({"role": "user", "parts": ["Invalid JSON. Please return ONLY valid JSON."]})
                continue

            print(f"\n[ROUTER STEP {i+1}]: {plan['thought']}")

            if plan['action'] == "final_answer":
                print(f"\n[ROUTER]: Task Complete.")
                return plan['content']

            elif plan['action'] == "call_agent":
                # Execute A2A Call
                agent_response = await self.call_agent(plan['agent_name'], plan['content'])

                snippet = str(agent_response)[:200].replace('\n', ' ')
                print(f"   <<< [A2A RESPONSE]: {snippet}...")

                # Add context to history
                messages.append({"role": "model", "parts": [clean_text]})
                messages.append({"role": "user", "parts": [f"Result from {plan['agent_name']}: {agent_response}"]})

        return "Error: Maximum turns reached without final answer."

# 3. Initialize the Orchestrator
router = RouterOrchestrator(a2a_client)
print("Router Orchestrator Initialized and ready.")

Router Orchestrator Initialized and ready.


In [35]:
router = RouterOrchestrator(a2a_client)
print("Router Orchestrator Initialized and ready to accept queries.")

Router Orchestrator Initialized and ready to accept queries.


## Testing A2A Communication

Demonstrating the A2A communication flow with comprehensive logging that tracks routing decisions, agent actions, and multi-agent coordination patterns.

### Customer Data Agent Testing

Testing direct communication with the Customer Data Agent to verify MCP tool integration and data retrieval capabilities.

In [21]:
async def test_customer_data():
    response = await a2a_client.create_task(
        'http://127.0.0.1:10020',  # correct
        'Look up customer with id 1'
    )
    print(response)

asyncio.run(test_customer_data())

2025-11-19 17:06:32,776 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7af56e3681d0>, 240.68955879)])']
connector: <aiohttp.connector.TCPConnector object at 0x7af56e79ecf0>


[CUSTOMER_DATA_AGENT]: Calling get_customer for ID 1
```json
{
  "customer": {
    "id": 1,
    "name": "John Doe",
    "email": "john.doe@example.com",
    "phone": "+1-555-0101",
    "status": "active",
    "created_at": "2025-11-17 21:35:02",
    "updated_at": "2025-11-17 21:35:02"
  }
}
```


### Support Agent Testing

Testing the Support Agent's ability to handle customer service queries and demonstrate intelligent routing when customer context is needed.

In [22]:
async def test_support_agent():
    response = await a2a_client.create_task(
        'http://localhost:10021',  # correct
        'A customer says they cannot log in. What should they do?'
    )
    print("Support Agent Response:\n", response)

asyncio.run(test_support_agent())

Support Agent Response:
 [SUPPORT_AGENT]: Responding to user's login issue.

I can help with that! First, try resetting your password by clicking the "Forgot Password" link on the login page. If you're still having trouble after that, let me know, and I can assist further.



### Router Agent Testing

Testing the Router's orchestration capabilities by coordinating between multiple agents for queries requiring both data retrieval and support assistance.

In [26]:
# async def test_router_agent():
#     response = await a2a_client.create_task(
#         'http://localhost:10040',
#         'Update customer 1 email to new-email@example.com'
#     )
#     print("Router Agent Response:\n", response)

# asyncio.run(test_router_agent())

Router Agent Response:
 ROUTE_TO_CUSTOMER_DATA_AGENT



## Assignment Test Scenarios

Demonstrating the three required coordination patterns from the assignment.

### Scenario 1: Task Allocation

**Query**: "I need help with my account, customer ID 1"

**Expected A2A Flow**:
1. Router receives query
2. Router → Customer Data Agent: Get customer info for ID 1
3. Customer Data Agent fetches data via MCP
4. Customer Data Agent → Router: Returns customer data
5. Router → Support Agent: Handle support with customer context
6. Support Agent generates response
7. Router returns final response

In [29]:
query_1 = "Get customer information for ID 1"

result_1 = await router.process_query(query_1)
print(f"\nFINAL OUTPUT SCENARIO 1:\n{result_1}")


 USER QUERY: Get customer information for ID 1

[ROUTER STEP 1]: The user is asking for customer information, and they have provided a specific customer ID. This requires accessing customer data. Therefore, I should use the customer_data agent.
   >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID 1 ```json {"customer": {     "id": 1,     "name": "John Doe",     "email": "john.doe@example.com",     "phone": "+1-555-0101",     "status": "active"...

[ROUTER STEP 2]: I have retrieved the customer information from the customer_data agent. Now I should present this information to the user.

[ROUTER]: Task Complete.

FINAL OUTPUT SCENARIO 1:
Here is the customer information for ID 1:

Name: John Doe
Email: john.doe@example.com
Phone: +1-555-0101
Status: active
Created At: 2025-11-17 21:35:02
Updated At: 2025-11-17 21:35:02


In [ ]:
# async def test_router_agent():
#     response = await a2a_client.create_task(
#         'http://localhost:10040',
#         "Get customer information for ID 1"
#     )
#     print("Router Agent Response:\n", response)

# asyncio.run(test_router_agent())

### Scenario 2: Negotiation/Escalation

**Query**: "I want to cancel my subscription but I'm having billing issues"

**Expected A2A Flow**:
1. Router detects multiple intents (cancellation + billing)
2. Router → Support Agent: Initial assessment
3. Support Agent → Router: Requests customer context
4. Router → Customer Data Agent: Get customer/billing info
5. Router coordinates response between agents
6. Final response addresses both concerns

In [30]:
# Scenario 2: Negotiation/Escalation
# Expected: Router sees billing issue -> calls Data Agent for info -> Calls Support agent with context
query_2 = "I want to cancel my subscription but I'm having billing issues. (My ID is 1)"

result_2 = await router.process_query(query_2)
print(f"\nFINAL OUTPUT SCENARIO 2:\n{result_2}")


 USER QUERY: I want to cancel my subscription but I'm having billing issues. (My ID is 1)

[ROUTER STEP 1]: The user wants to cancel their subscription AND is having billing issues. I need to address both. First, I'll ask the support agent about the cancellation policy. Then, I'll ask the customer data agent to get the customer's billing history to understand the billing issues.
   >>> [A2A CALL] Connecting to support_agent at http://localhost:10021...
   <<< [A2A RESPONSE]: [SUPPORT_AGENT]: Answering general question about cancellation policy.  Our cancellation policy allows you to cancel your subscription at any time. The cancellation will be effective at the end of you...

[ROUTER STEP 2]: Okay, I have the cancellation policy. Now I need to investigate the billing issues. I need to fetch the customer's details and billing history from the customer_data agent using the customer ID.
   >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUS

In [ ]:
# async def test_router_agent():
#     response = await a2a_client.create_task(
#         'http://localhost:10040',
#         "I want to cancel my subscription but I'm having billing issues"
#     )
#     print("Router Agent Response:\n", response)

# asyncio.run(test_router_agent())

### Scenario 3: Multi-Step Coordination

**Query**: "Show me all active customers who have open tickets"

**Expected A2A Flow**:
1. Router decomposes query into sub-tasks
2. Router → Customer Data Agent: Get all active customers
3. Customer Data Agent → Router: Returns customer list
4. Router → Customer Data Agent: Get ticket history for each customer
5. Router filters for open tickets
6. Router synthesizes final report

In [37]:
# Scenario 3: Multi-Step Coordination
# Expected: Router calls list_customers -> Router loops through results to get history -> Router filters
query_3 = "Show me the names of all active customers who have closed tickets."

result_3 = await router.process_query(query_3)
print(f"\nFINAL OUTPUT SCENARIO 3:\n{result_3}")


 USER QUERY: Show me the names of all active customers who have closed tickets.

[ROUTER STEP 1]: The user is asking for a list of active customers with closed tickets. I need to use the customer_data agent to first get a list of active customers, then get the ticket history for all of those customers, and finally filter to show the names of the active customers with closed tickets.
   >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling list_customers with status active. ```json {"customer": [{"id": 4, "name": "Alice Williams", "email": "alice.w@techcorp.com", "phone": "+1-555-0104", "status": "active",...


2025-11-19 17:16:16,922 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7af565ba84d0>, 823.661566511)])']
connector: <aiohttp.connector.TCPConnector object at 0x7af565c066f0>



[ROUTER STEP 2]: Now that I have the list of active customers, I need to get the ticket history for all of them. I will then filter this ticket history to only include those customers with closed tickets.
   >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling get_customer_history for IDs 4, 5, 6, 7, 9, 10, 11, 2, 1, 12, 14, 15 ```json {"customer_histories": [{"customer_id": 4, "history": [{"id": 2, "customer_id": 4, "issue": ...

[ROUTER STEP 3]: I now have the ticket histories for all active customers. I need to filter the results to only show the names of active customers who have at least one closed (resolved) ticket.

[ROUTER]: Task Complete.

FINAL OUTPUT SCENARIO 3:
The following active customers have closed tickets:
- Charlie Brown
- George Miller
- Isaac Newton
- Jane Smith
- Laura Martinez
- Michael Scott


In [ ]:
# async def test_router_agent():
#     response = await a2a_client.create_task(
#         'http://localhost:10040',
#         "What's the status of all high-priority tickets for active customers?"
#     )
#     print("Router Agent Response:\n", response)

# asyncio.run(test_router_agent())